In [1]:
from glob import glob
import json

In [2]:
files = sorted(glob('function-call-complex*/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

49069

In [3]:
def check_function(s):
    name = s.get('name', s.get('nama'))
    if name is None:
        return
    if 'parameters' not in s:
        return
    if 'properties' not in s['parameters']:
        return

    properties = set(s['parameters']['properties'])

    success = False
    required = s['parameters'].get('required', s['parameters'].get('diperlukan'))
    if required:
        required = set(required)
        if len(required & properties) == len(required):
            success = True
    else:
        success = True
    
    if success:
        return name, properties
    
def check_functions(function_calls):
    functions = {}

    if isinstance(function_calls, dict):
        o = check_function(data['e']['function_call'])
        if o is not None:
            functions[o[0]] = o[1]

    elif isinstance(function_calls, str):

        try:
            for s in function_calls.split('\n\n'):
                try:
                    s = eval(s)
                except:
                    continue

                o = check_function(s)
                if o is not None:
                    functions[o[0]] = o[1]
        except:
            s = eval(function_calls)
            if isinstance(s, dict):
                s = [s]
            for s_ in s:
                o = check_function(s_)
                if o is not None:
                    functions[o[0]] = o[1]
                
    return functions
    
rejected = []
with open('function-calls-complex.jsonl', 'w') as fopen_l:
    for f in files:
        try:
            with open(f) as fopen:
                data = json.load(fopen)

            rs = [data]

            for r in rs:

                try:
                    function_calls = r['function_call']
                    functions = check_functions(function_calls)

                    if not len(functions):
                        continue
                    
                    conversations = r.get('conversations', r.get('perbualan'))
                    if len(conversations) % 2 != 0:
                        conversations = conversations[:-1]

                    pairs = []
                    for i in range(0, len(conversations), 2):
                        try:
                            splitted = conversations[i + 1].get('content', conversations[i + 1].get('kandungan'))
                            if splitted is None:
                                continue
                            if '> ' in splitted:
                                splitted = splitted.split('> ')
                                splitted = splitted[1]
                            
                            splitted = splitted.replace('true', 'True').replace('false', 'False')
                            
                            failed = True
                            try:
                                splitted = eval(splitted)
                                failed = False
                            except:
                                pass
                            
                            if failed:
                                try:
                                    splitted = eval(splitted + '}')
                                    failed = False
                                except:
                                    pass
                            
                            if failed:
                                try:
                                    splitted = eval(splitted.replace("'{", '{') + '}')
                                except:
                                    continue
                                    
                            # print('spllitted', splitted, splitted['name'])
                            
                            name = splitted.get('name', splitted.get('nama'))
                            arguments = splitted.get('arguments', splitted.get('argument', splitted.get('argumen')))
                            if arguments is None:
                                arguments = splitted.get('hujah', splitted.get('hujahan'))
                            if arguments is None:
                                arguments = splitted.get('arahan', splitted.get('parameter'))
                            
                            if name not in functions:
                                continue
                                
                            if isinstance(arguments, str):
                                arguments = eval(arguments)
                                
                            arguments_keys = set(arguments.keys())
                            if len(functions[name] & arguments_keys) == len(arguments_keys):
                                splitted['arguments'] = arguments
                                c = [
                                    {
                                        'role': 'user',
                                        'content': conversations[i]['content'],
                                    },
                                    {
                                        'role': 'assistant',
                                        'content': f'<functioncall> {str(splitted)}'
                                    }
                                    ]
                                pairs.extend(c)

                        except Exception as e_splitted:
                            # print('e_splitted', e_splitted, conversations[i + 1], splitted)
                            pass

                    if function_calls and len(pairs):
                        d = {
                            'function_call': function_calls,
                            'conversations': pairs,
                        }
                        fopen_l.write(f'{json.dumps(d)}\n')
                        fopen_l.flush()
                except Exception as e_loop:
                    print('e_loop', e_loop, r)
                    rejected.append((e_loop, r))

        except Exception as e:
            print(e, data)

e_loop 'str' object has no attribute 'get' {'function_call': '"HantarPesan"', 'conversations': [{'role': 'user', 'content': 'Hantar pesanan untuk dua piza vegetarian dan sebotol air mineral'}, {'role': 'assistant', 'content': "<functioncall> {'name': 'HantarPesan', 'arguments': {'pesanan': [{'jenis': 'vegetarian', 'jumlah': 2}, {'jenis': 'air mineral', 'jumlah': 1}]}}"}]}
e_loop 'str' object has no attribute 'get' {'function_call': '"Achchi Nulu Bowe Kow"', 'conversations': [{'role': 'user', 'content': 'Apa sudah jadi kat Seri Kembangan? Kenapa jalan sesak sangat?'}, {'role': 'assistant', 'content': '<functioncall> {"name": "Achchi Nulu Bowe Kow", "arguments": {"triplets": [{"subject": "Seri Kembangan", "predicate": "tersumbat", "object": "jalan"}, {"subject": "jalan", "predicate": "sebabkan", "object": "sesak"}]}}'}]}
e_loop 'str' object has no attribute 'get' {'function_call': '"{name: \\"KunciGanda\\", description: \\"gabung semua inti \'KunciGanda\' dengan segala parameter yang dip

In [4]:
!wc -l function-calls-complex.jsonl

24986 function-calls-complex.jsonl


In [5]:
ls = []
with open('function-calls-complex.jsonl') as fopen:
    for l in fopen:
        ls.append(json.loads(l))
        if len(ls) > 100:
            break

In [6]:
ls[-1]

{'function_call': '{"name": "HR","description": "berikan maklum balas menggunakan kaedah `HR` dengan semua parameter yang diperlukan dengan jenis yang betul","parameters": {"$defs": {"Data": {"properties": {"pegawai": {"type": "string"}, "maklumat": {"type": "string"}}, "required": ["pegawai", "maklumat"], "type": "object"}}, "properties": {"feedbacks": {"items": {"$ref": "#/$defs/Data"}, "type": "array"}}, "required": ["feedbacks"], "type": "object"}}',
 'conversations': [{'role': 'user',
   'content': 'Pegawai yang melakukan kesalahan melaporkan diri kepada ketua bahagian HR.'},
  {'role': 'assistant',
   'content': "<functioncall> {'name': 'HR', 'arguments': {'feedbacks': [{'pegawai': 'Pegawai', 'maklumat': 'melakukan kesalahan'}, {'pegawai': 'Pegawai', 'maklumat': 'laporkan diri kepada ketua bahagian HR'}]}}"}]}

In [7]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='function-calls-complex.jsonl',
    path_in_repo='function-calls-complex.jsonl',
    repo_id='mesolitica/chatgpt-malay-function-call',
    repo_type='dataset',
)

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


function-calls-complex.jsonl:   0%|          | 0.00/27.8M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/chatgpt-malay-function-call/blob/main/function-calls-complex.jsonl'